In [1]:
from pathlib import Path

from cleantext import clean
import dataset
import requests
import pycountry
from tqdm import tqdm

In [2]:
import gettext
german = gettext.translation('iso3166', pycountry.LOCALES_DIR, languages=['de'])
german.install()
# _("Germany")

In [3]:
auth = tuple(Path('secrets.txt').read_text().split()[1:])

In [4]:
db = dataset.connect('sqlite:///test.db')


In [11]:
tab_loc = db['locations_final']
tab_incidents = db['incidents']

In [12]:
duplicates = []
statement = 'SELECT * FROM locations_final GROUP BY longitude, latitude having count(*) > 1'
for row in db.query(statement):
    dupli = list(tab_loc.find(latitude=row['latitude'], longitude=row['longitude']))
    new_items = []
#     for x in dupli:
#         num = len(list(tab_incidents.find(latitude=row['latitude'], longitude=row['longitude'])))
#         new_items.append([x, num])
    if len(dupli) > 1:
        duplicates.append(dupli)

In [13]:
len(duplicates)

23

In [21]:
duplicates[6]

[OrderedDict([('id', 657),
              ('latitude', 51.83606),
              ('longitude', 12.24284),
              ('postal_code', None),
              ('street', None),
              ('house_number', None),
              ('district', None),
              ('city', None),
              ('county', 'Dessau-Roßlau'),
              ('state', 'Sachsen-Anhalt'),
              ('country', 'Deutschland')]),
 OrderedDict([('id', 658),
              ('latitude', 51.83606),
              ('longitude', 12.24284),
              ('postal_code', '06844'),
              ('street', None),
              ('house_number', None),
              ('district', None),
              ('city', 'Dessau-Roßlau'),
              ('county', 'Dessau-Roßlau'),
              ('state', 'Sachsen-Anhalt'),
              ('country', 'Deutschland')])]

In [ ]:
def fix_duplicates(duplicates):
    nums = [x[1] for x in duplicates]
    print(nums)
    max_nums = max(nums)
    if nums.count(max_nums) == 1:
        max_index = nums.index(max_nums)
        winner = duplicates[max_index][0]
        for x in duplicates:
            if x[0] == winner:
                continue
            print('merging', x[0]['subdivisions_complete'], winner['subdivisions_complete'])
#             for old in tab_incidents.find(subdivisions_complete=x[0]['subdivisions_complete']):
#                 merged = {**old, **winner}
#                 tab_incidents.update(merged, ['id'])
            
#             tab_loc.delete(subdivisions_complete=x[0]['subdivisions_complete'])

In [ ]:
for d in duplicates:
    fix_duplicates(d)

In [22]:
duplicates[0]

[[OrderedDict([('id', 271),
               ('latitude', 52.5046),
               ('longitude', 9.45725),
               ('subdivisions_complete',
                "[['city', 'Dresden - Innere Neustadt'], ['state', 'Sachsen'], ['country', 'Deutschland']]"),
               ('subdivisions', "[['', 'Dresden - Innere Neustadt']]")]),
  1],
 [OrderedDict([('id', 394),
               ('latitude', 52.5046),
               ('longitude', 9.45725),
               ('subdivisions_complete',
                "[['city', 'Dresden, Innere Neustadt'], ['state', 'Sachsen'], ['country', 'Deutschland']]"),
               ('subdivisions', "[['', 'Dresden, Innere Neustadt']]")]),
  1]]

In [42]:
r = requests.post('https://geocode.app.vis.one/', auth=auth, json={'locations': [{'location':'Auerbach, Landkreis Vogtlandkreis, Deutschland'}]})

In [43]:
r.json()

{'locations': [{'latitude': 48.40026,
   'location': 'Auerbach, Landkreis Vogtlandkreis, Deutschland',
   'longitude': 10.67081}]}

In [41]:
r.json()

{'locations': [{'latitude': 49.28948,
   'location': 'Auerbach, Landkreis Vogtlandkreis, Sachsen, Deutschland',
   'longitude': 10.66038}]}

In [39]:
r.json()

{'locations': [{'latitude': 49.28948,
   'location': 'Auerbach, Vogtlandkreis, Sachsen, Deutschland',
   'longitude': 10.66038}]}

In [38]:
duplicates[0]

[OrderedDict([('id', 41),
              ('latitude', 49.28948),
              ('longitude', 10.66038),
              ('location_string',
               'Auerbach, Vogtlandkreis, Sachsen, Deutschland'),
              ('subdivisions',
               "[['', 'Auerbach'], ['Landkreis', 'Vogtlandkreis']]")]),
 OrderedDict([('id', 1050),
              ('latitude', 49.28948),
              ('longitude', 10.66038),
              ('location_string', 'Ostsachsen, Sachsen, Deutschland'),
              ('subdivisions', "[['', 'Ostsachsen']]")]),
 OrderedDict([('id', 1131),
              ('latitude', 49.28948),
              ('longitude', 10.66038),
              ('location_string',
               'Rosenbach, Vogtlandkreis, Sachsen, Deutschland'),
              ('subdivisions',
               "[['', 'Rosenbach'], ['Landkreis', 'Vogtlandkreis']]")]),
 OrderedDict([('id', 1148),
              ('latitude', 49.28948),
              ('longitude', 10.66038),
              ('location_string', 'Sachsen, Sac